In [ ]:
#import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,StratifiedKFold,StratifiedShuffleSplit,ShuffleSplit
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler,RobustScaler,PolynomialFeatures,StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline 
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,StackingRegressor,VotingRegressor
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.base import TransformerMixin
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
import seaborn as sns

In [ ]:
#install the catboost library for modelling
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.9 MB/s eta 0:00:00


In [ ]:
#import the regressor model
from catboost import CatBoostRegressor

In [ ]:
#mount google drive for file management
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#read the necessary datasets
train=pd.read_csv("/content/drive/MyDrive/nfts_train.csv",index_col=0)
test=pd.read_csv("/content/drive/MyDrive/nfts_predict.csv",index_col=0)
collections=pd.read_csv("/content/drive/MyDrive/collections.csv",index_col=0)
collections_twitter=pd.read_csv("/content/drive/MyDrive/collections_twitter_stats.csv",index_col=0)

In [ ]:
#open the train data
#we are to predict the last sale price at the last sale date
#I checked and found that nft_ids aren't repeated for a given collection, so we don't have a sense of the previous prices for that NFT.
train.head(30)

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date,last_sale_price
global_index,,,,,,,,
21928,0,49,2.000000,NaN,NaN,NaN,2022-11,1.266732
32621,0,45,97.077216,NaN,NaN,NaN,2021-08,1.200357
32622,1,45,139.453086,NaN,NaN,NaN,2022-08,3.180572
32623,2,45,127.753445,NaN,NaN,NaN,2021-08,3.502910
32624,3,45,112.315719,NaN,NaN,NaN,2022-09,4.009567
32625,4,45,179.395901,NaN,NaN,NaN,2021-08,2.001691
32626,5,45,152.067286,NaN,NaN,NaN,2021-10,2.502283
32627,6,45,139.320270,NaN,NaN,NaN,2022-08,3.191687
32628,7,45,107.340146,NaN,NaN,NaN,2021-08,1.996968


In [ ]:
#check for negative last_sale_price values
#there are none apparently
for i in range(len(train)):
  count=0
  if train.iloc[i,-1]<0:
    count+=1
    train.drop([i], inplace = True)
print(count)

0


In [ ]:
#checking the distribution of the seller fees
collections['seller_fees'].value_counts()

500.0     13
1000.0    11
750.0      7
900.0      6
800.0      3
250.0      3
550.0      1
200.0      1
700.0      1
Name: seller_fees, dtype: int64

In [ ]:
#a quick description of the numerical columns 
train.describe()

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_price
count,309910.000000,309910.000000,309910.000000,80480.000000,80480.000000,80480.000000,3.099100e+05
mean,6738.162024,30.507160,1437.092531,1.000000,5630.164737,11255.829672,2.162269e+00
std,5977.542096,18.331668,4303.585892,0.152825,3812.126968,3452.713804,1.275132e+01
min,0.000000,0.000000,1.000000,0.626738,1.000000,5555.000000,6.485360e-08
25%,2453.000000,13.000000,106.587184,0.906820,2515.750000,9928.000000,4.077085e-03
50%,5138.000000,34.000000,170.797030,0.978289,5030.000000,10000.000000,8.103344e-03
75%,8720.750000,46.000000,289.614129,1.059866,8207.250000,15000.000000,3.252062e-01
max,29898.000000,60.000000,90354.072248,3.663833,15555.000000,15555.000000,1.024002e+03


In [ ]:
#checking the distribution of the train's collection ids
test['collection_id'].value_counts()

39    48257
35    25492
2     20216
27    10000
22    10000
29     9999
28     9999
53     9999
3      9061
5      8638
42     7145
47     5295
41     3781
10     3332
Name: collection_id, dtype: int64

In [ ]:
#getting a feel of the collections data
collections.head(20)

,collection_id,total_supply,creation_date,verification_status,n_of_traits,contract_type,seller_fees,platform_fees,openrarity_enabled,has_website,has_own_twitter,has_discord,has_medium
0,19,1000,2021-04,verified,4,own,500.0,250,False,False,True,True,True
1,45,8888,2021-07,verified,5,own,500.0,250,False,False,True,True,False
2,44,10000,2022-04,verified,8,own,500.0,250,True,False,True,True,False
3,53,9999,2022-07,verified,9,own,900.0,250,False,False,True,True,False
4,6,9602,2021-06,verified,8,own,NaN,250,False,False,True,True,False
5,59,9508,2021-11,verified,1,own,500.0,250,False,False,True,True,False
6,49,1,2022-04,verified,2,own,1000.0,250,False,False,True,True,False
7,60,420,2022-06,verified,1,own,900.0,250,False,False,True,True,False
8,23,9998,2021-08,verified,14,own,500.0,250,False,False,True,True,False
9,29,9999,2022-05,verified,10,own,750.0,250,False,False,True,False,False


In [ ]:
#let's see the twitter data for every collection
collections_twitter.head(10)

,collection_id,n_tweets_in_range,avg_likes,avg_replies,avg_retweets,min_likes,min_replies,min_retweets,max_likes,max_replies,max_retweets
0,0,2,183.000000,8.500000,41.000000,123,3,6,243,14,76
1,1,16,36.375000,3.562500,13.937500,0,0,0,125,21,34
2,2,1,15.000000,9.000000,1.000000,15,9,1,15,9,1
3,3,526,1.178707,0.306084,2.655894,0,0,0,21,8,13
4,4,26,8.153846,0.884615,1.153846,0,0,0,30,6,12
5,5,19,7.842105,5.157895,4.000000,0,0,0,67,75,55
6,6,12,0.000000,0.000000,111.833333,0,0,31,0,0,463
7,7,12,0.000000,0.000000,111.833333,0,0,31,0,0,463
8,8,40,8.075000,3.450000,3.250000,0,0,0,91,96,88
9,10,3,4.000000,0.666667,1.333333,0,0,1,12,2,2


In [ ]:
#sort the dataframe first by collection ID and then by last_sale_date
#I was thinking I could come up with a feature to track overall collection price. I didn't have enough time to explore this option
new_train=train.sort_values(by=['collection_id', 'last_sale_date'])

In [ ]:
#check the shape of the collections_twitter data
collections_twitter.shape

(48, 11)

In [ ]:
#merge the collection and collection_twitter data to the train data
new_train_merged=new_train.merge(collections, left_on='collection_id', right_on='collection_id')
new_train_merged=new_train_merged.merge(collections_twitter, left_on='collection_id', right_on='collection_id')

In [ ]:
#I wanted to know if any nfts within a given collection were repeated
new_train_merged_exploratory=new_train_merged[(new_train_merged['nft_id'] == 0) & (new_train_merged['collection_id']==1)]

In [ ]:
new_train_merged_exploratory

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date
global_index,,,,,,,
0,0,29,176.364208,NaN,NaN,NaN,2022-06
1,1,29,207.209228,NaN,NaN,NaN,2022-05
2,2,29,157.293414,NaN,NaN,NaN,2022-10
3,3,29,173.372185,NaN,NaN,NaN,2022-06
4,4,29,190.077584,NaN,NaN,NaN,2022-06
...,...,...,...,...,...,...,...
9994,9994,29,180.295144,NaN,NaN,NaN,2022-07
9995,9995,29,182.265215,NaN,NaN,NaN,2022-10
9996,9996,29,155.042216,NaN,NaN,NaN,2022-10


In [ ]:
#open the merged data
new_train_merged.head(50)

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date,last_sale_price,total_supply,creation_date,...,n_tweets_in_range,avg_likes,avg_replies,avg_retweets,min_likes,min_replies,min_retweets,max_likes,max_replies,max_retweets
0,4625,0,199.542391,NaN,NaN,NaN,2022-04,1.105469,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
1,4640,0,359.482872,NaN,NaN,NaN,2022-04,1.050154,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
2,4651,0,226.240443,NaN,NaN,NaN,2022-04,1.156019,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
3,4652,0,922.680050,NaN,NaN,NaN,2022-04,0.004764,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
4,4653,0,145.204965,NaN,NaN,NaN,2022-04,1.171465,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
5,4655,0,127.287314,NaN,NaN,NaN,2022-04,1.184444,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
6,4656,0,182.421699,NaN,NaN,NaN,2022-04,1.224305,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
7,4657,0,246.861979,NaN,NaN,NaN,2022-04,1.166481,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
8,4690,0,466.617600,NaN,NaN,NaN,2022-04,1.172737,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76
9,4691,0,76.539933,NaN,NaN,NaN,2022-04,1.004766,6501,2022-04,...,2,183.0,8.5,41.0,123,3,6,243,14,76


In [ ]:
#open the test data
test.head(5)

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date
global_index,,,,,,,
0,0,29,176.364208,NaN,NaN,NaN,2022-06
1,1,29,207.209228,NaN,NaN,NaN,2022-05
2,2,29,157.293414,NaN,NaN,NaN,2022-10
3,3,29,173.372185,NaN,NaN,NaN,2022-06
4,4,29,190.077584,NaN,NaN,NaN,2022-06


In [ ]:
#run similar daya transformations for the test data 
new_test=test.sort_values(by=['collection_id', 'last_sale_date'])
new_test=new_test.merge(collections, left_on='collection_id', right_on='collection_id')
new_test=new_test.merge(collections_twitter, left_on='collection_id', right_on='collection_id')

In [ ]:
#open merged tes data
new_test.head(5)

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date,total_supply,creation_date,verification_status,...,n_tweets_in_range,avg_likes,avg_replies,avg_retweets,min_likes,min_replies,min_retweets,max_likes,max_replies,max_retweets
0,20010,2,152.754156,NaN,NaN,NaN,2022-04,20279,2021-07,approved,...,1,15.0,9.0,1.0,15,9,1,15,9,1
1,20011,2,190.435932,NaN,NaN,NaN,2022-04,20279,2021-07,approved,...,1,15.0,9.0,1.0,15,9,1,15,9,1
2,20012,2,119.280035,NaN,NaN,NaN,2022-04,20279,2021-07,approved,...,1,15.0,9.0,1.0,15,9,1,15,9,1
3,20013,2,154.429693,NaN,NaN,NaN,2022-04,20279,2021-07,approved,...,1,15.0,9.0,1.0,15,9,1,15,9,1
4,20016,2,324.101268,NaN,NaN,NaN,2022-04,20279,2021-07,approved,...,1,15.0,9.0,1.0,15,9,1,15,9,1


In [ ]:
#converting the creation date and last sale date values to date_time and subsequently to timestamps so i can compute the difference between the two
to_time=pd.to_datetime(new_train_merged["creation_date"])
new_train_merged["creation_date"]=to_time
new_train_merged['creation_date']= [datetime.timestamp(i)/(60*60*24) for i in to_time]
to_time=pd.to_datetime(new_train_merged["last_sale_date"])
new_train_merged["last_sale_date"]=to_time
new_train_merged['last_sale_date']= [datetime.timestamp(i)/(60*60*24) for i in to_time]
new_train_merged['days_elapsed']=new_train_merged['last_sale_date']-new_train_merged['creation_date']

In [ ]:
#Apparently, there are data points with negative 'days_elapsed' values. This simoly cannot be
new_train_merged['days_elapsed'].value_counts()
#so i dropped them
for i in range(len(new_train_merged)):
  if new_train_merged.loc[i, 'days_elapsed']<0:
    new_train_merged.drop([i], inplace=True)

 0.0      46770
 31.0     26025
 153.0    19571
 30.0     15809
 426.0    15518
          ...  
 638.0       14
-59.0         8
-62.0         7
-245.0        2
-181.0        1
Name: days_elapsed, Length: 65, dtype: int64

In [ ]:
#confirm that they have been dropped
new_train_merged['days_elapsed'].value_counts()

0.0      46770
31.0     26025
153.0    19571
30.0     15809
426.0    15518
         ...  
549.0      126
610.0      105
577.0       56
607.0       54
638.0       14
Name: days_elapsed, Length: 61, dtype: int64

In [ ]:
#needed to confrim that my time values will get properly converted
datetime.timestamp(pd.to_datetime(['2021-07'])[0])

1625097600.0

In [ ]:
#do for the test data what we have done for the train
to_time=pd.to_datetime(new_test["creation_date"])
new_test["creation_date"]=to_time
new_test['creation_date']= [datetime.timestamp(i)/(60*60*24) for i in to_time]
to_time=pd.to_datetime(new_test["last_sale_date"])
new_test["last_sale_date"]=to_time
new_test['last_sale_date']= [datetime.timestamp(i)/(60*60*24) for i in to_time]
new_test['days_elapsed']=new_test['last_sale_date']-new_test['creation_date']

In [ ]:
#view the merged test data
new_test.head()

,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date,total_supply,creation_date,verification_status,...,avg_likes,avg_replies,avg_retweets,min_likes,min_replies,min_retweets,max_likes,max_replies,max_retweets,days_elapsed
0,20010,2,152.754156,NaN,NaN,NaN,19083.0,20279,18809.0,approved,...,15.0,9.0,1.0,15,9,1,15,9,1,274.0
1,20011,2,190.435932,NaN,NaN,NaN,19083.0,20279,18809.0,approved,...,15.0,9.0,1.0,15,9,1,15,9,1,274.0
2,20012,2,119.280035,NaN,NaN,NaN,19083.0,20279,18809.0,approved,...,15.0,9.0,1.0,15,9,1,15,9,1,274.0
3,20013,2,154.429693,NaN,NaN,NaN,19083.0,20279,18809.0,approved,...,15.0,9.0,1.0,15,9,1,15,9,1,274.0
4,20016,2,324.101268,NaN,NaN,NaN,19083.0,20279,18809.0,approved,...,15.0,9.0,1.0,15,9,1,15,9,1,274.0


In [ ]:
#create a last_Sale_price column for the new_test data so that it can be easily separated from the combined train-test data
new_test['last_sale_price']=[1 for i in range(new_test.shape[0])]

In [ ]:
#concatenate train and test data to avoid repetitive transformations
data=pd.concat([new_train_merged,new_test])

In [ ]:
#generate new features
data['difference_fees']=data['seller_fees']-data['platform_fees']
data['difference_rank']=data['rarity_score']-data['openrarity_score']
data['total fees']=data['seller_fees']+data['platform_fees']
data['total_avg']=data['avg_likes']+data['avg_retweets']+data['avg_replies']
data['difference_likes']=data['max_likes']-data['min_likes']
data['difference_retweets']=data['max_retweets']-data['min_retweets']
data['difference_replies']=data['max_replies']-data['min_replies']

In [ ]:
#convert the categorical features to numeric 
for col in data.columns:
    if data[col].dtype=="object":
      data[col]=pd.factorize(data[col])[0]

In [ ]:
#get train and test data
train_df=data.loc[data['last_sale_price']!=1]
test_df=data.loc[data['last_sale_price']==1]

In [ ]:
#split the train set to get an eval_df for local validation
train_df, eval_df = train_test_split(train_df, test_size=0.30, random_state=42, shuffle=True )

In [ ]:
#instantiate model
model=CatBoostRegressor()

In [ ]:
#define target variable 
y=train_df['last_sale_price']

In [ ]:
#drop target column to get train_df ready for training
train_df.drop(['last_sale_price'], axis=1, inplace=True)

In [ ]:
X=train_df

In [ ]:
# fit model
model.fit(X, y)

Learning rate set to 0.095789
0:	learn: 12.3702311	total: 47.3ms	remaining: 47.3s
1:	learn: 11.7384210	total: 92.8ms	remaining: 46.3s
2:	learn: 11.1886370	total: 138ms	remaining: 45.8s
3:	learn: 10.7135229	total: 212ms	remaining: 52.9s
4:	learn: 10.3036562	total: 257ms	remaining: 51.1s
5:	learn: 9.9516699	total: 299ms	remaining: 49.6s
6:	learn: 9.6575545	total: 346ms	remaining: 49s
7:	learn: 9.4005637	total: 385ms	remaining: 47.7s
8:	learn: 9.1819818	total: 424ms	remaining: 46.7s
9:	learn: 8.9958557	total: 467ms	remaining: 46.2s
10:	learn: 8.8355110	total: 516ms	remaining: 46.4s
11:	learn: 8.6964116	total: 561ms	remaining: 46.2s
12:	learn: 8.5805255	total: 605ms	remaining: 45.9s
13:	learn: 8.4846718	total: 652ms	remaining: 45.9s
14:	learn: 8.4013908	total: 695ms	remaining: 45.6s
15:	learn: 8.3293697	total: 750ms	remaining: 46.1s
16:	learn: 8.2723442	total: 790ms	remaining: 45.7s
17:	learn: 8.2155999	total: 831ms	remaining: 45.3s
18:	learn: 8.1679127	total: 881ms	remaining: 45.5s
19:	le

In [ ]:
#evaluate model
y_true=eval_df['last_sale_price']
eval_df.drop(['last_sale_price'], axis=1, inplace=True)
X_test=eval_df
y_model=model.predict(X_test)

In [ ]:
#import error metric function
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error

In [ ]:
mse(y_model, y_true)

45.11760855205905

In [ ]:
test_ind=test_df.index

In [ ]:
submission1=pd.DataFrame(index=test_ind)

In [ ]:
test.shape

(181214, 7)

In [ ]:
test_df.shape

(181214, 38)

In [ ]:
submission1['last_sale_price']=model.predict(test_df)

In [ ]:
submission1.to_csv("CATBOOST_SOLUTION1.csv", index=False)

In [ ]:
submission1.last_sale_price.value_counts()

0.278259    48235
0.250158    25489
0.472524    20213
0.604615     9058
0.492287     8635
            ...  
0.492395        1
0.278376        1
0.278719        1
0.278012        1
0.451753        1
Name: last_sale_price, Length: 246, dtype: int64

In [ ]:
# further improvements
# generate more features
# hyperparameter tuning